## Interacting with ArcGIS Online using the ArcGIS API for Python!

<br>

The ArcGIS Python API allows you to connect to your ArcGIS Online Organization through Python. 
<br>
Just like accessing ArcGIS Online through a web browser, you need to login to access your data.
<br>
Data in ArcGIS Online is stored as feature layers and tables. 
<br>
These have similar properties to feature classes and feature tables that are held in geodatabases.
<br>
Featuresets are similar to the ArcPy Cursors we have worked with in arcpy. These are used to update the feature services.
<br>
Link to arcgis Python API documentation: https://developers.arcgis.com/python/api-reference/

<b> Before diving into the Python, lets all sign into ArcGIS online using our credentials<br>
    Using the following link: https://www.arcgis.com/index.html </b>

In [ ]:
import os
import arcgis

from IPython.display import display

This is the first time we have seen the "from" keyword in the import cell.
<br>
The from statement allows us to import only the modules we are interested in.
<br>
This can help us avoid importing large packages when we only need a few modules from them.

In [ ]:
# Make sure your credentials are signed into AGOL before running this cell
mygis = arcgis.GIS('pro')

In [ ]:
mygis

In [ ]:
mygis.users.me

In [ ]:
# this is how you can log in to your AGOL from outside ArcGIS Pro

# this will cause an error since I do not have my passwor included here!
private_gis = arcgis.GIS(r"https://usfca.maps.arcgis.com/", "EricS_BayGeo", "password")

In [ ]:
map_sample = mygis.map("Walnut Creek, CA")
map_sample

Lets check out the documentation for the parameters of "map"<br>
https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis

In [ ]:
mygis.map("Walnut Creek, CA", 11, mode='3D')

Lets add some data to the map!

To do this, lets look into the content manager method:
<br>
https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?highlight=content%20search#arcgis.gis.ContentManager
<br>
https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?highlight=content%20search#arcgis.gis.ContentManager.search

In [ ]:
items = mygis.content.search('Walnut Creek', item_type='feature layer', outside_org=True)

In [ ]:
items

In [ ]:
for item in items:
    display(item)

In [ ]:
# how can we get the first item in the set of items?
cal_fire_polygon = items[0]

In [ ]:
# feature layer collection
cal_fire_polygon

In [ ]:
# There can actually be multiple layers within one feature layer collection
# to get the layers in a list, we would use the .layers method
cal_fire_polygon.layers

In [ ]:
map_sample = mygis.map("Walnut Creek, CA")

In [ ]:
# recreate our map sample
map_sample.add_layer(cal_fire_polygon)
map_sample.zoom_to_layer(cal_fire_polygon.layers[0])

In [ ]:
map_sample

### Before uploading data to ArcGIS Online, lets talk about the data types we will see

1. Feature
    - An individual record or row from a feature class, shapefile, or feature layer. An entity located in space (some kind of geometry) with a set of properties.
2. Feature Layer
    - In the ArcGIS API for python, a feature layer is special object representing a group of features of one type (point, polygon, line). For example, the
        first layer in the cal_fire_polygon feature collection is a individual feature layer. It is important to understand the difference between a FeatureLayer
        and a feature collection because they have different capabilities.
3. Table
    - A table with no spatial information, containing records of information. These can also be found within a feature collection.
4. Feature Collection
    - A collection of feature layers and tables is a feature layer collection. You can't do much with a feature collection:
      think of it as a list of Feature Layers and Tables, an object we can use to organize feature layers and tables.
5. Feature Set
    - A special ArcGIS API for Python object. Feature sets are created from a FeatureLayer Python object. They act very similair to how an ArcPy cursor behaves, except instead of 
        using local data like a feature class, it is scripted against a online FeatureLayer.


### Lets upload our own data to ArcGIS Online

In [ ]:
import os
import arcgis

mygis = arcgis.GIS('pro')

In [ ]:
project_path = os.getcwd()

# need to change the zip file name to something unique!
your_name = ''
wc_restaurants_full_path = os.path.join(project_path, f'wc_restaurants_{}.zip')

In [ ]:
wc_restaurants_full_path

In [ ]:
item_properties = {"type": "Shapefile",
                   "title" : "Walnut Creek Restaurants",
                    "tags": "Restaurants, Walnut Creek, CA",
                   "description" : "Restaurants in Walnut Creek, CA. Created 2022 from Python",
                   "commentsEnabled" : False}

The .add method will just upload the shapefile or gdb to arcgis online. It will not publish it as a feature layer

In [ ]:
wc_restaurants_item_shp = mygis.content.add(item_properties, data=wc_restaurants_full_path)

In [ ]:
wc_restaurants_item_shp

Lets move over to arcgis online to see what this looks like

To Publish it as a feature service, we must use the .publish method

In [ ]:
wc_restaurants_feature_layer = wc_restaurants_item_shp.publish()

Lets move over to arcgis online to see what this looks like
<br>
Lets also add it to our map pane to check out the data further

<b> Great! Now that we have the shapefile as a feature layer, we can delete the shapefile file </b>

In [ ]:
wc_restaurants_item_shp.delete()

### Accessing and Editing the Data

In order to view, query, and edit the data that's on AGOL, a feature layer object within python must be made.

These can be created either using the search method, or using the rest service URL.

After the feature layer object is initiated, a feature set can be created (similair to a cursor). 
<br>
A feature set enables the user to make changes
and then set these changes as permanent back in the original feature layer

Link to the docs for featurelayers:
https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer

In [ ]:
# lets search for the data that we just created!

local_items = mygis.content.search('Walnut Creek Restaurants', item_type='feature layer', outside_org=False)

for i in local_items:
    display(i)

In [ ]:
# to get a layer as a feature layer python object, we will use the following function:

arcgis.features.FeatureLayer(local_items[0])

We want to always be explicit, so the search method may not be the best way to programmtically create a feature layer in python

The best way is by calling directly to the URL where the feature layer is hosted, <br>
that way you know you are getting the correct feature layer.

Lets go grab the URL

In [ ]:
# input your own URL here!
Walnut_Creek_Restaurants_rest_url = r'https://services5.arcgis.com/wXYNaciObHUosEnt/arcgis/rest/services/wc_restaurants_erics/FeatureServer/0'

fl = arcgis.features.FeatureLayer(Walnut_Creek_Restaurants_rest_url)

In [ ]:
fl

Now lets create a feature set out of the Feature Layer!

In [ ]:
fs = fl.query()

In [ ]:
fs

In [ ]:
for row in fs:
    print(row)

In [ ]:
row

In [ ]:
#how would we access the attributes dictionary within the feature set?

row.geometry

In [ ]:
# temp_dict = {'age': 25, }

In [ ]:
row.geometry['spatialReference']

In [ ]:
row.attributes['name']

.query() has some handy extra parameters that we can use to parse down some of the data we return!

In [ ]:
fl.query(return_count_only=True)

In [ ]:
fl.query(where = "cuisine = 'mexican'", return_count_only=True)

Remember ArcPy Cursors? Well Feature Sets are similair!
<br>
In fact, I think it's good practice to think of the two as the same, just used for different types of data:<br>
<b>-Cursors are for local data <br>
-Feature Sets are for AGOL data <br></b>

In [ ]:
fs = fl.query()

for row in fs:
    print(row)

In [ ]:
# First lets try using a feature set to replace all blank cuisine's with a value
fs = fl.query()

for row in fs:
    cuisine_value = row.attributes['cuisine']
    if cuisine_value == ' ':
        row.attributes['cuisine'] = 'missing'

In [ ]:
# update feature layer
update_fl = fl.edit_features(updates=fs)

Additionally, Instead of using feature sets to make these changes to the feature layer, <br>
We can actually use similair logic that we use within ArcGIS Pro, when we select for attributes <br>
and then calculate fields:

In [ ]:
fl.calculate(where="FID < 10",
                   calc_expression={"field": "Rating", "value" : 1})